### Spajanje podataka

#### Ulazni podaci:
- ```Rasporedi (Ponedeljak.xlsx, Utorak.xlsx,...)```
- ```Plan99.xls```
- ```Realizacija.xlsx```

#### Proces spajanja
1. prolaziti kroz sve pročitane termine iza rasporeda
2. pronalaziti odgovarajuće entitete koji su ranije parsirani iz plana i realizacije
    1. pronađen, dodeli ga
    2. nije pronađen, kreiraj ga

In [1]:
from model_parser import *
from parser_utils import map_class_lengths
from collections import defaultdict

#### Učitavanje svih fajlova

In [2]:
# ucitano na osnovu rasporeda
raspored = RasporedTermin.read_list_from_file('3_raspored')
raspored_izmene = raspored.copy()
# ucitano na osnovu realizacije i dodatnih fajlova + prosireno planom
schedule = MeetingSchedule.read_entity_from_file('2_svi_fajlovi_plan')
schedule_izmene = MeetingSchedule.read_entity_from_file('2_svi_fajlovi_plan')

### Dopunjavanje pročitanog rasporeda

- raspored pročitan iz ```Ponedeljak.xlsx, Utorak.xlsx,...``` dopunjavati podacima pročitanih iz ```Plan99.xls```

#### Izbacivanje termina koji su vezani za predavanja Japanskog

- Japanski jezik

In [3]:
def remove_japanski(
        raspored: list[RasporedTermin]
) -> list[RasporedTermin]:
    raspored = [termin for termin in raspored if termin.predmet != 'Japanski jezik']
    return raspored

In [4]:
raspored_izmene = remove_japanski(raspored)

#### Pronalaženje katedre
- **nepostojeće oznake katedri**: {162, 100, 211, 212, 213}
    - 100 je Japanski jezik, biće izbačen
    - 162 je greška u unosu, treba da bude **192**
    - 211 je Odsek za automatiku, geomatiku i upravljanje sistemima
        - mapiraćemo na katedru za automatiku: **2111**
    - 212 je Odsek za primenjene računarske nauke i informatiku
        - mapiraćemo na katedru za informatiku: **216**
    - 213 je Odsek za računarsku tehniku i računarske komunikacije
        - mapiraćemo na katedru za računarsku tehniku: **2131**

In [5]:
def extract_predavac_names(
        raspored: list[RasporedTermin]
) -> list[RasporedTermin]:
    for termin in raspored:
        predavaci = []
        multiple_names = termin.predavac.split(',')
        multiple_names = [name.split() for name in multiple_names]
        for name in multiple_names:
            predavaci.append(tuple(name))
        termin.predavaci_imena = predavaci
    return raspored

In [6]:
def find_predavac(
        name: tuple[str],
        predavaci: list[Predavac]
) -> Predavac:
    # da li je predavac nov, ako je nov -> missing je
    if name[0] == 'Nov' or name[0] == 'Novi' or name[0] == 'Predavač':
        return None
    
    # prezime ime
    if len(name) == 2:
        prezime = name[0]
        ime = name[1]
        predavac = next((x for x in predavaci if x.ime == ime and x.prezime == prezime), None)
        if predavac:
            return predavac
        return None
    # vise prezimena vise imena
    # ako postoji bar jedno prezime plus to ime
    if len(name) == 3 and name[1] not in ['mr', 'dr']:
        prezime = name[0]
        ime = name[-1]
        predavac = next((x for x in predavaci if ime in x.ime and prezime in x.prezime), None)
        if predavac:
            return predavac
        return None
    
    # prezime titula ime
    if len(name) == 3 or (len(name) == 4 and name[1] in ['mr', 'dr']):
        prezime = name[0]
        ime = name[2]
        predavac = next((x for x in predavaci if x.ime == ime and x.prezime == prezime), None)
        if predavac:
            # kad ga vratis, setuj mu titulu
            return predavac
        return None
    # vise prezimena titula ime
    if len(name) >= 4:
        prezime = ' '.join(name[:-2])
        ime = name[-1]
        predavac = next((x for x in predavaci if x.ime == ime and x.prezime == prezime), None)
        if predavac:
            # kad ga vratis, setuj mu titulu
            return predavac
        # vise prezimena sa crticom
        else:
            prezime = name[0] + '-' + name[1]
            predavac = next((x for x in predavaci if x.ime == ime and x.prezime == prezime), None)
            if predavac:
                # kad ga vratis, setuj mu titulu
                return predavac
            return None

In [7]:
def find_missing_katedre(
        raspored: list[RasporedTermin], 
        katedre: list[Katedra],
        predavaci: list[Predavac]
) -> set[int]:
    missing = set()
    for termin in raspored:
        oznaka = int(termin.oznakaKatedre)
        katedra = next((x for x in katedre if x.oznaka == oznaka), None)
        if katedra is None:
            missing.add(oznaka)
    return missing

In [8]:
def find_similar_katedra(
        oznaka: int,
        katedre: list[Katedra]
) -> Katedra:
    katedra = None
    if oznaka == 162:
        katedra = next(x for x in katedre if x.oznaka == 192)
    elif oznaka == 211:
        katedra = next(x for x in katedre if x.oznaka == 2111)
    elif oznaka == 212:
        katedra = next(x for x in katedre if x.oznaka == 216)
    elif oznaka == 213:
        katedra = next(x for x in katedre if x.oznaka == 2131)
    return katedra

In [9]:
def map_katedre(
        raspored: list[RasporedTermin], 
        katedre: list[Katedra],
        predavaci: list[Predavac]
) -> list[RasporedTermin]:
    raspored = extract_predavac_names(raspored)
    for termin in raspored:
        oznaka = int(termin.oznakaKatedre)
        katedra = next((x for x in katedre if x.oznaka == oznaka), None)
        if katedra is not None:
            termin.oznakaKatedre_id = katedra.id
        else:
            found_predavac = False
            for idx, predavac_name in enumerate(termin.predavaci_imena):
                predavac = find_predavac(predavac_name, predavaci)
                # ako postoji predavac, namapiraj njegovu katedru
                if predavac is not None:
                    found_predavac = True
                    termin.oznakaKatedre_id = predavac.orgJedinica
                    break
            if not found_predavac:
                # ne postoji predavac u sistemu, niti katedra
                # namapiraj slicnu katedru
                katedra = find_similar_katedra(oznaka, katedre)
                termin.oznakaKatedre_id = katedra.id
    return raspored

In [10]:
raspored_izmene = map_katedre(raspored_izmene, schedule.katedraList, schedule.predavacList)

#### Pronalaženje predmeta i studijskog programa

- ukupno **41** termin čiji naziv ne postoji u planu 
- ukupno **6** termina čiji predmeti čija je šifra struke potpuno različita, ali ime postoji:
    - 1.
        - raspored: 20MSSEMS1030
        - plan:     20EMS1030
        - naziv predmeta: Softver za FN sisteme u realnim uslovima rada
    - 2.
        - raspored: 20MSSEMS1030
        - plan:     20EMS1030
        - naziv predmeta: Specijalne električne instalacije
    - 3.
        - raspored: 99E2191
        - plan:     21E21690
        - naziv predmeta: Metodologija brzog razvoja softvera
    - 4.
        - raspored: 99E2277
        - plan:     13E22570
        - naziv predmeta: Web bazirani merno-akvizicaioni sistemi


In [11]:
def map_predmeti_stud_programi(
        raspored: list[RasporedTermin], 
        predmeti: list[Predmet]
) -> list[RasporedTermin]:
    for termin in raspored:
        predmet = next((x for x in predmeti if (x.naziv == termin.predmet and x.sifraStruke == termin.sifraStruke)), None)
        if predmet is None:
            # u rasporedu postoje oznake tipa 99E2151
            # dok se u planu sve oznake zavrsavaju 0
            predmet_slican = next((x for x in predmeti if (x.naziv == termin.predmet and termin.sifraStruke[:-1] in x.sifraStruke)), None)
            if predmet_slican is not None:
                predmet = predmet_slican
                termin.sifraStruke = predmet_slican.sifraStruke
            else:
                predmet_naziv = next((x for x in predmeti if x.naziv == termin.predmet), None)
                if predmet_naziv is not None:
                    predmet = predmet_naziv
                    termin.sifraStruke = predmet_naziv.sifraStruke
                else:
                    continue
        termin.predmet_id = predmet.id
        termin.studProgram_id = predmet.studijskiProgram
    return raspored

In [12]:
raspored_izmene = map_predmeti_stud_programi(raspored_izmene, schedule.predmetList)

In [13]:
raspored_izmene[1]

RasporedTermin(oznakaKatedre_id='c19f3ca3-2319-4653-b531-8fdbf3a9d640', predmet_id='56a3c084-c1a1-48a6-bf29-696a76c58f14', studProgram_id='66c36462-e5f4-4440-b425-8b7d277eb4a7', predavac_id=None, ostaliPredavaci=None, sifraStruke='99E2110', semestar='I', predmet='Matematička analiza 1', tipNastave='Pred.', studGrupa='SVI', predavac='Ralević dr Nebojša', oznakaKatedre='531', ukupnoStud='240', trajanje=8, predavaci_imena=[('Ralević', 'dr', 'Nebojša')])

### Dopunjavanje informacija o predmetu iz realizacije informacijama iz rasporeda

- ```Realizacija``` **nema** dobre informacije o tipovima vežbi i broju časova
- ```Plan``` **nema** dobre informaciju o tipovima vežbi i broju časova
- ```Raspored``` **ima** dobre informacije o tipovima vežbi i broju časova
- => dopunjavamo parsirane podatke podacima iz rasporeda

In [14]:
def extract_tipovi_vezbi_for_predmet(
        raspored: list[RasporedTermin],
) -> dict[set]:
    predmet_tipovi = defaultdict(lambda: set())
    for termin in raspored:
        if termin.tipNastave == 'Pred.':
            continue
        predmet_tipovi[termin.predmet_id].add(termin.tipNastave)
    return predmet_tipovi

In [15]:
def count_termini_for_tip_vezbi(
        raspored: list[RasporedTermin]
) -> dict[list[int]]:
    # koliko puta jedna grupa ima jedan tip vezbi iz jednog predmeta u toku nedelje i koliko taj termin traje
    # (stud_program_id, predmet_id, tip_vezbi, grupa) -> [broj_pojavljivanja, trajanje]
    broj_termina_za_tip_vezbi = defaultdict(lambda: [0, 0])
    for termin in raspored:
        if termin.tipNastave == 'Pred.':
            continue
        # ne brojimo ako nismo prethodno pronasli predmet ni u planu ni u realizaciji
        if termin.predmet_id is None:
            continue
        broj_termina_za_tip_vezbi[(termin.studProgram_id, termin.predmet_id, termin.tipNastave, termin.studGrupa)][0] += 1
        broj_termina_za_tip_vezbi[(termin.studProgram_id, termin.predmet_id, termin.tipNastave, termin.studGrupa)][1] = termin.trajanje
    return broj_termina_za_tip_vezbi

In [16]:
def map_casovi_for_tip_vezbi(
        grupa: str,
        broj_termina: int,
        trajanje_termina_15: int
):
    # ako je 3, poseban slucaj dvoje vezanih vezbi od 45 minuta bez pauze
    if trajanje_termina_15 != 3:
        # ostale, umanji za 1 (broj podeoka od 15 minuta)
        trajanje_termina_15 -= 1
    trajanje_termina_casovi = map_class_lengths(trajanje_termina_15)
    # vezbe se odrzavaju svake druge nedelje => broj_casova_vezbi / 2
    if '(' in grupa and '-' not in grupa:
        trajanje_termina_casovi = int(trajanje_termina_casovi / 2)
    # ukupan broj casova vezbi je: trajanje jednog termina * broj termina
    if trajanje_termina_casovi is None:
        print(trajanje_termina_15)
        trajanje_termina_casovi = 3
    broj_casova_vezbi = trajanje_termina_casovi * broj_termina
    return broj_casova_vezbi

In [17]:
def map_tipovi_vezbi_for_predmet(
        raspored: list[RasporedTermin],
        predmeti: list[Predmet]
):
    broj_termina_za_tip_vezbi = count_termini_for_tip_vezbi(raspored)
    for oznake, termini in broj_termina_za_tip_vezbi.items():
        predmet_id = oznake[1]
        tip_vezbi = oznake[2]
        grupa = oznake[3]
        broj_termina = termini[0]
        trajanje_termina_15 = termini[1]
        # izracunaj broj casova vezbi
        broj_casova_vezbi = map_casovi_for_tip_vezbi(grupa, broj_termina, trajanje_termina_15)
        # predmetu dodaj informaciju o broju casova vezbi
        predmet = next(x for x in predmeti if x.id == predmet_id)
        if tip_vezbi == 'aud.vežbe':
            predmet.brojCasovaAud = broj_casova_vezbi
        if tip_vezbi == 'rač.vežbe':
            predmet.brojCasovaRac = broj_casova_vezbi
        if tip_vezbi == 'lab.vežbe':
            predmet.brojCasovaLab = broj_casova_vezbi
    return predmeti

In [18]:
schedule_izmene.predmetList = map_tipovi_vezbi_for_predmet(raspored_izmene, schedule_izmene.predmetList)

### Pronalaženje predavača

- Dodavanje predavačima titula
- Kreiranje nedostajućih predavača (novi saradnici i osobe koje su menjale zvanje)

In [19]:
import uuid

In [20]:
def map_predavac_titula(
        name: tuple[str],
        predavac: Predavac
) -> Predavac:
    if 'mr' in name:
        predavac.titula = 'mr'
    elif 'dr' in name:
        predavac.titula = 'dr'
    return predavac

In [21]:
def create_predavac(
        name: tuple[str],
        oznaka: int,
        katedra_id: str
) -> Predavac:
    # novi
    titula = ''
    if name[0] == 'Nov' or name[0] == 'Novi' or name[0] == 'Predavač':
        prezime = 'Saradnik'
        ime = 'Nov'
    # prezime ime
    elif len(name) == 2:
        prezime = name[0]
        ime = name[1]
    # vise prezimena titula ime
    elif len(name) >= 3 and ('mr' in name or 'dr' in name):
        prezime = ' '.join(name[:-2])
        titula = name[-2]
        ime = name[-1]
    # vise prezimena
    elif len(name) >= 3:
        prezime = name[0] + ' ' + name[1]
        ime = name[2]
    return Predavac(str(uuid.uuid4()), oznaka, ime, prezime, False, False, katedra_id, titula)

In [22]:
def map_predavaci(
        raspored: list[RasporedTermin],
        predavaci: list[Predavac]
) -> tuple[list[RasporedTermin], list[Predavac]]:
    # oznaka novih predavaca
    oznaka = 9999
    raspored = extract_predavac_names(raspored)
    # za svaki termin
    for termin in raspored:
        # prodji svakog predavaca
        for idx, predavac_name in enumerate(termin.predavaci_imena):
            # pronadji predavaca
            predavac = find_predavac(predavac_name, predavaci)
            if predavac:
                # dodaj titulu predavacu
                predavac = map_predavac_titula(predavac_name, predavac)
            else:
                # napravi novog
                predavac = create_predavac(predavac_name, oznaka, termin.oznakaKatedre_id)
                predavaci.append(predavac)
                oznaka -= 1
            if idx == 0:
                # ako je prvi, postavi kao glavnog
                termin.predavac_id = predavac.id
            else:
                # ako nije, postavi u ostale
                if not termin.ostaliPredavaci:
                    termin.ostaliPredavaci = []
                termin.ostaliPredavaci.append(predavac.id)
    return raspored, predavaci

In [23]:
raspored_izmene, schedule_izmene.predavacList = map_predavaci(raspored_izmene, schedule_izmene.predavacList)

In [24]:
print(raspored_izmene[1])
print(schedule_izmene.predmetList[1])

RasporedTermin(oznakaKatedre_id='c19f3ca3-2319-4653-b531-8fdbf3a9d640', predmet_id='56a3c084-c1a1-48a6-bf29-696a76c58f14', studProgram_id='66c36462-e5f4-4440-b425-8b7d277eb4a7', predavac_id='4b230fe2-d150-4450-a06f-f56e5f92c863', ostaliPredavaci=None, sifraStruke='99E2110', semestar='I', predmet='Matematička analiza 1', tipNastave='Pred.', studGrupa='SVI', predavac='Ralević dr Nebojša', oznakaKatedre='531', ukupnoStud='240', trajanje=8, predavaci_imena=[('Ralević', 'dr', 'Nebojša')])
Predmet(id='438af07c-ab2b-441c-a0b9-3a07800adb68', oznaka='RG018', plan=20, naziv='Osnovi informacionih tehnologija za računarsku animaciju', godina=1, semestar='Z', brojCasovaPred=2, studijskiProgram='294a1492-8fb7-4737-abfd-13b096e560f9', brojCasovaVezbe=2, sifraStruke='99C5310', tipoviNastave='', brojCasovaAud=-1, brojCasovaLab=-1, brojCasovaRac=3)


### Dodavanje predmeta prostorijama posebne namene

- neke prostorije imaju posebnu namenu
- samo određeni predmeti mogu u njima da se održavaju
- zapisivanje koji predmeti smeju da se održavaju u tim prostorijama

In [25]:
def hemija_posebni_predmeti(
        predmeti: list[Predmet]
) -> list[str]:
    posebni_predmeti_ids = []
    predmet = next(x for x in predmeti if x.oznaka == 'Z600' and x.naziv == 'Hemijski fenomeni u inženjerstvu')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'Z102' and x.naziv == 'Hemijski fenomeni u inženjerstvu zaštite životne sredine')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'Z151' and x.naziv == 'Hemijski fenomeni u mašinstvu')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'F103' and x.naziv == 'Hemija u grafičkom inženjerstvu')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'ZR222' and x.naziv == 'Fizički i hemijski parametri radne sredine')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'Z153' and x.naziv == 'Hemijski fenomeni u inženjerstvu zaštite na radu')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'Z507' and x.naziv == 'Fizičko hemijski principi')
    posebni_predmeti_ids.append(predmet.id)
    return posebni_predmeti_ids

In [26]:
def deformisanje_posebni_predmeti(
        predmeti: list[Predmet]
) -> list[str]:
    posebni_predmeti_ids = []
    predmet = next(x for x in predmeti if x.oznaka == 'P2401' and x.naziv == 'Napredne metode tehnologije plastičnog deformisanja')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'P3501' and x.naziv == 'Projektovanje alata za plastiku')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'P3503A' and x.naziv == 'Savremeni obradni sitemi za preradu plastike')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'P3402' and x.naziv == 'Fizička i fazna stanja polimera')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'M3203' and x.naziv == 'Tehnologija mašinogradnje')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'P207' and x.naziv == 'Tehnologija plastičnog deformisanja')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'P3403' and x.naziv == 'Tehnologija oblikovanja plastike')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'P2413K' and x.naziv == 'Projektovanje alata za deformisanje')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'F504I0' and x.naziv == '3D štampa')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'P2407K' and x.naziv == 'Aditivne tehnologije')
    posebni_predmeti_ids.append(predmet.id)
    predmet = next(x for x in predmeti if x.oznaka == 'BMIM4B' and x.naziv == 'Tehnologije oblikovanja biomedicinskih materijala')
    posebni_predmeti_ids.append(predmet.id)
    return posebni_predmeti_ids

In [27]:
def prostorije_posebne_namene(
        prostorije: list[Prostorija],
        predmeti: list[Predmet]
) -> list[Prostorija]:
    hemija = next(x for x in prostorije if x.oznaka == 'Hemija')
    hemija2 = next(x for x in prostorije if x.oznaka == 'Hemija 2')
    hemija_predmeti = hemija_posebni_predmeti(predmeti)
    hemija.odobreniPredmeti = hemija_predmeti
    hemija2.odobreniPredmeti = hemija_predmeti

    mi_d4 = next(x for x in prostorije if x.oznaka == 'MI D4')
    mi_d4d = next(x for x in prostorije if x.oznaka == 'MI D4-D')
    deformisanje_predmeti = deformisanje_posebni_predmeti(predmeti)
    mi_d4.odobreniPredmeti = deformisanje_predmeti
    mi_d4d.odobreniPredmeti = deformisanje_predmeti

    return prostorije

In [28]:
schedule_izmene.prostorijaList = prostorije_posebne_namene(schedule_izmene.prostorijaList, schedule_izmene.predmetList)

### Usklađivanje informacija o predmetima

- svi predmeti sa istim planom i istom oznakom predmeta treba da imaju isti broj časova vežbi i predavanja

In [29]:
def match_predmet_broj_vezbi(
        predmeti: list[Predmet]
) -> list[Predmet]:
    predmeti_group = defaultdict(list[Predmet])
    for predmet in predmeti:
        predmeti_group[(predmet.oznaka, predmet.plan)].append(predmet)
    
    for _, predmet_lista in predmeti_group.items():
        max_aud = max(predmet.brojCasovaAud for predmet in predmet_lista)
        max_lab = max(predmet.brojCasovaLab for predmet in predmet_lista)
        max_rac = max(predmet.brojCasovaRac for predmet in predmet_lista)
        for predmet in predmet_lista:
            predmet.brojCasovaAud = max_aud
            predmet.brojCasovaLab = max_lab
            predmet.brojCasovaRac = max_rac
    predmeti = [predmet for predmet_lista in predmeti_group.values() for predmet in predmet_lista]
    return predmeti

In [30]:
schedule_izmene.predmetList = match_predmet_broj_vezbi(schedule_izmene.predmetList)

### Zapisivanje svih izmena i uvezivanja u fajl

- zapisivanje rasporada
- zapisivanje svih izvučenih informacija
- zapisivanje pojedinačno svake liste podataka

In [31]:
ReadWrite.write_to_file(raspored_izmene, '4_raspored_spojen')
ReadWrite.write_to_file(schedule_izmene, '4_svi_fajlovi_spojeni')

In [32]:
empty = ['semestar', 'meetingList', 'meetingAssignmentList']
attrs = [a for a in vars(schedule_izmene) if a not in empty]

for attr in attrs:
    list = getattr(schedule_izmene, attr)
    ReadWrite.write_to_file(list, '4_' + attr, '../out_data/pojedinacno/')